In [201]:
import os
import time
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt

# Dash modules
import dash
import dash_table
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import dash_html_components as html
from jupyter_dash import JupyterDash
import plotly.express as px
import plotly.graph_objects as go

# Set up jupyter proxy
JupyterDash.infer_jupyter_proxy_config() 

%matplotlib inline

In [31]:
# Getting all file paths

path = r'assets' # use your path
all_files = glob.glob(path + "/*.csv")

# Creating list to append all ticker dfs to
li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

# Concat all ticker dfs 
stock_df = pd.concat(li, axis=0, ignore_index=True)

stock_df['Date'] = pd.to_datetime(stock_df['Date'])

# Creating Moving Average Technical Indicator
# Using this aritcle https://towardsdatascience.com/building-a-comprehensive-set-of-technical-indicators-in-python-for-quantitative-trading-8d98751b5fb

stock_df['SMA_5'] = stock_df.groupby('ticker')['Close'].transform(lambda x: x.rolling(window = 5).mean())
stock_df['SMA_15'] = stock_df.groupby('ticker')['Close'].transform(lambda x: x.rolling(window = 15).mean())
stock_df['SMA_ratio'] = stock_df['SMA_15'] / stock_df['SMA_5']

# Bollinger bands
stock_df['15MA'] = stock_df.groupby('ticker')['Close'].transform(lambda x: x.rolling(window=15).mean())
stock_df['SD'] = stock_df.groupby('ticker')['Close'].transform(lambda x: x.rolling(window=15).std())
stock_df['upperband'] = stock_df['15MA'] + 2*stock_df['SD']
stock_df['lowerband'] = stock_df['15MA'] - 2*stock_df['SD']

available_indicators1 = list(stock_df['ticker'].unique())

In [252]:
title_style = {'display': 'inline-block',
               'textAlign':'left',
               'verticalAlign':'center',
               'lineHeight':2.5,
               'height':75,
               'border': 'thick black solid',\
               'width':'85%',
               'backgroundColor': 'rgb(212, 150, 18)'}

title_link_style = {'display': 'inline-block',
               'textAlign':'center',
               'vertical-align':'center',
               'lineHeight':5,
               'height':75,
               'width':'10%',
               'float':'right',
               'border': 'thick black solid',\
               'fontSize':12,
               'backgroundColor': 'rgb(212, 150, 18)'}


tab_style = {'display': 'inline-block',
             'textAlign':'center',
             'verticalAlign':'top',
             'width':'15%',
             'border': 'thin lightgrey solid',
             'height':20}

blank_tab_style = {'display': 'inline-block',
                   'width':'15%',
                   'textAlign':'center',
                   'verticalAlign':'top',
                   'float':'right',
                   'border': 'thin lightgrey solid',
                   'fontSize':10,
                   'height':20}

ticker_style = {'display': 'inline-block',
                'textAlign':'center',
                'vertical-align':'top',
                'width':'20%',
                'float':'right',
                'border':'thin lightgrey solid',
                'height':300}

chart_style = {'display':'inline-block',
               'textAlign':'center',
               'vertical-align':'top',
               'width':'60%',
               'float':'middle',
               'textAlign':'center',
               'border':'thin lightgrey solid',
               'height':300}

portfolio_style = {'display': 'inline-block',
                'textAlign':'center',
                'vertical-align':'top',
                'width':'20%',
                'float':'left',
                'border':'thin lightgrey solid',
                'height':300}

chart_style_b = {'display':'inline-block',
               'textAlign':'center',
               'vertical-align':'top',
               'width':'60%',
               'float':'middle',
               'textAlign':'center',
               'border':'thin lightgrey solid',
               'height':200}

portfolio_style_b = {'display': 'inline-block',
                'textAlign':'left',
                'fontSize':12,
                'vertical-align':'top',
                'width':'20%',
                'float':'left',
                'border':'thin lightgrey solid',
                'height':200}



#                'vertical-align':'middle',

In [253]:
app = JupyterDash(__name__,suppress_callback_exceptions=True)

server = app.server

app.layout = html.Div([
    
    html.Div([
        html.H1('Dashboard Title',style=title_style),
        html.H3('Dashboard Info Link',style=title_link_style)
        ]),
    
    dcc.Tabs(id='tabs-example', value='tab-1', children=[
        dcc.Tab(label='Portfolio Performance', value='tab-1'),
        dcc.Tab(label='Model Performance', value='tab-2'),
        dcc.Tab(label='Tab three', value='tab-3')
    ],style={'width':'50%','height':50}),
    html.Div(id='tabs-example-content')
])

@app.callback(dash.dependencies.Output('tabs-example-content', 'children'),
              dash.dependencies.Input('tabs-example', 'value'))
def render_content(tab):
    if tab == 'tab-1':
        return html.Div([
            
                html.Div([
                    dcc.Dropdown(id='ticker',
                    options=[{'label': i, 'value': i} for i in available_indicators1],
                    value='CSCO') # the default is code_module AAA
                    ]),
                
                html.Div([
                    html.H2('Portfolio Performance',style=portfolio_style),
                    dcc.Graph(id='price_chart',style=chart_style),
                    news_info
                    ]),
                    
                html.Div([
                    html.H2('Other Portfolio Statistics',style=portfolio_style_b),
                    html.H2('News Info',style=chart_style_b),
                    ],style={'height':200})
    
                ])
                
    elif tab == 'tab-2':
        return html.Div([
            html.H3('Tab content 2')
        ])
    
# Callback to connect input(s) to output(s)
@app.callback(dash.dependencies.Output('price_chart','figure'),
    [dash.dependencies.Input('ticker','value')])

# Step 3: Define the graph with plotly express
def update_ticker(ticker):
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(x=stock_df[stock_df['ticker']==ticker]['Date'],
                             y=stock_df[stock_df['ticker']==ticker]['Close'],
                            line={"color": "#228B22"},
                            mode="lines"))

    fig.update_layout(title_text=f'{ticker} Closing Price',title_x=0.5,
                         template="plotly_dark",font=dict(size=10),xaxis_showgrid=False,
                         yaxis_title="Closing Price",margin={"r": 20, "t": 35, "l": 20, "b": 10})

    return fig

if __name__ == '__main__':
    app.run_server(debug=True,port=8051)

Dash app running on http://127.0.0.1:8051/


### Notes
1. How do I set a relative height
2. Could add ticker - https://community.plotly.com/t/strip-ticker-label/48348/6
3. Need right link for articles pulled for news - use regex to pull id and article title into URL


In [16]:
import FinNews as fn

seeking_alpha = fn.SeekingAlpha(topics=['$AAPL'], save_feeds=True)

In [36]:
apple_news = seeking_alpha.get_news()

URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>

In [119]:
apple_news[0]

{'title': 'Apple stock a buy after closing up on Fed day?',
 'title_detail': {'type': 'text/plain',
  'language': None,
  'base': 'https://seekingalpha.com/api/sa/combined/AAPL.xml',
  'value': 'Apple stock a buy after closing up on Fed day?'},
 'links': [{'rel': 'alternate',
   'type': 'text/html',
   'href': 'https://seekingalpha.com/symbol/AAPL/news?source=feed_symbol_AAPL'}],
 'link': 'https://seekingalpha.com/symbol/AAPL/news?source=feed_symbol_AAPL',
 'id': 'https://seekingalpha.com/MarketCurrent:3707326',
 'guidislink': False,
 'published': 'Thu, 17 Jun 2021 10:11:04 -0400',
 'published_parsed': time.struct_time(tm_year=2021, tm_mon=6, tm_mday=17, tm_hour=14, tm_min=11, tm_sec=4, tm_wday=3, tm_yday=168, tm_isdst=0),
 'sa_author_name': 'Kim Khan',
 'media_thumbnail': [{'url': ''}],
 'href': '',
 'sa_picture': '',
 'sa_symbol': 'AAPL',
 'sa_company_name': 'Apple Inc.',
 'sa_stock': '',
 'topic': '$AAPL'}

In [120]:
# apple_news[0]['id'].

'https://seekingalpha.com/MarketCurrent:3707326'

In [20]:
pd.DataFrame.from_dict(apple_news)

,title,title_detail,links,link,id,guidislink,published,published_parsed,sa_author_name,media_thumbnail,href,sa_picture,sa_symbol,sa_company_name,sa_stock,topic
0,Apple stock a buy after closing up on Fed day?,"{'type': 'text/plain', 'language': None, 'base...","[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://seekingalpha.com/symbol/AAPL/news?sour...,https://seekingalpha.com/MarketCurrent:3707326,False,"Thu, 17 Jun 2021 10:11:04 -0400","(2021, 6, 17, 14, 11, 4, 3, 168, 0)",Kim Khan,[{'url': ''}],,,AAPL,Apple Inc.,,$AAPL
1,Apple iPhone: China Upside Surprise In May,"{'type': 'text/plain', 'language': None, 'base...","[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://seekingalpha.com/article/4435196-apple...,https://seekingalpha.com/Article:4435196,False,"Wed, 16 Jun 2021 15:12:54 -0400","(2021, 6, 16, 19, 12, 54, 2, 167, 0)",Paulo Santos,[{'url': ''}],,,AAPL,Apple Inc.,,$AAPL
2,House antitrust bill would prevent Apple from ...,"{'type': 'text/plain', 'language': None, 'base...","[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://seekingalpha.com/symbol/AAPL/news?sour...,https://seekingalpha.com/MarketCurrent:3707020,False,"Wed, 16 Jun 2021 14:30:51 -0400","(2021, 6, 16, 18, 30, 51, 2, 167, 0)",Brandy Betz,[{'url': ''}],,,IBM,International Business Machines Corporation,,$AAPL
3,"Apple Stock Forecast For 2025: A Slow Start, T...","{'type': 'text/plain', 'language': None, 'base...","[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://seekingalpha.com/article/4435098-apple...,https://seekingalpha.com/Article:4435098,False,"Wed, 16 Jun 2021 09:33:45 -0400","(2021, 6, 16, 13, 33, 45, 2, 167, 0)",Trading Places Research,[{'url': ''}],,,AAPL,Apple Inc.,,$AAPL
4,Apple reportedly struggling with healthcare pu...,"{'type': 'text/plain', 'language': None, 'base...","[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://seekingalpha.com/symbol/AAPL/news?sour...,https://seekingalpha.com/MarketCurrent:3706860,False,"Wed, 16 Jun 2021 08:58:28 -0400","(2021, 6, 16, 12, 58, 28, 2, 167, 0)",Brandy Betz,[{'url': ''}],,,AAPL,Apple Inc.,,$AAPL
5,Is Apple Stock Good For A Dividend Portfolio?,"{'type': 'text/plain', 'language': None, 'base...","[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://seekingalpha.com/article/4435082-apple...,https://seekingalpha.com/Article:4435082,False,"Wed, 16 Jun 2021 08:30:00 -0400","(2021, 6, 16, 12, 30, 0, 2, 167, 0)",Jonathan Weber,[{'url': ''}],,,AAPL,Apple Inc.,,$AAPL
6,Apple: If The Darkest Night Is Before The Dawn...,"{'type': 'text/plain', 'language': None, 'base...","[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://seekingalpha.com/article/4434974-apple...,https://seekingalpha.com/Article:4434974,False,"Tue, 15 Jun 2021 14:41:36 -0400","(2021, 6, 15, 18, 41, 36, 1, 166, 0)",Oleh Kombaiev,[{'url': ''}],,,AAPL,Apple Inc.,,$AAPL
7,Senate confirms sharp critic of Big Tech to FT...,"{'type': 'text/plain', 'language': None, 'base...","[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://seekingalpha.com/symbol/AAPL/news?sour...,https://seekingalpha.com/MarketCurrent:3706555,False,"Tue, 15 Jun 2021 13:09:16 -0400","(2021, 6, 15, 17, 9, 16, 1, 166, 0)",Jason Aycock,[{'url': ''}],,,GOOG,Alphabet Inc.,,$AAPL
8,"Apple, Google mobile ecosystems targeted in UK...","{'type': 'text/plain', 'language': None, 'base...","[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://seekingalpha.com/symbol/AAPL/news?sour...,https://seekingalpha.com/MarketCurrent:3706318,False,"Tue, 15 Jun 2021 07:18:14 -0400","(2021, 6, 15, 11, 18, 14, 1, 166, 0)",Brandy Betz,[{'url': ''}],,,AAPL,Apple Inc.,,$AAPL
9,Apple Is 68% Overvalued But These 4 Hyper-Grow...,"{'type': 'text/plain', 'language': None, 'base...","[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://seekingalpha.com/article/4434854-apple...,https://seekingalpha.com/Article:4434854,False,"Tue, 15 Jun 2021 04:42:05 -0400","(2021, 6, 15, 8, 42, 5, 1, 166, 0)",Dividend Sensei,[{'url': ''}],,,AAPL,Apple Inc.,,$AAPL


In [9]:
import sys
sys.executable

'/Users/tylerdrumheller/opt/anaconda3/bin/python'

In [169]:
# app = JupyterDash(__name__)

# server = app.server

news_style = {'fontSize':11,
              'color':'white',
              'textAlign':'left',
              'height':27,
              'margin':3,
              "lineHeight":1.5,
              'float':'middle'}

news_style_b = {'display': 'inline-block',
                'textAlign':'left',
                'vertical-align':'top',
                'width':'20%',
                'float':'right',
                'border':'thin lightgrey solid',
                'height':300,'margin':0,'overflowY':'scroll',
                'color':'white',
                'backgroundColor':'black',
                'borderRadius': '.4rem'}

news_info = html.Div([
        html.Div('News for AAPL',style={'backgroundColor':'gray'}),
        dbc.ListGroup(
            [
                dbc.ListGroupItem([html.Div([
                                        html.A(html.P(apple_news[0]['title'],style=news_style),
                                        href=(f"{apple_news[0]['link']}")),
                                        html.A(html.P(apple_news[0]['published'],style=news_style))
                                    ])
                                  ],color='gray'),
                dbc.ListGroupItem([html.Div([
                                        html.A(html.P(apple_news[1]['title'],style=news_style),
                                        href=(f"{apple_news[1]['link']}")),
                                        html.A(html.P(apple_news[1]['published'],style=news_style))
                                    ])
                                  ],color='gray'),
                dbc.ListGroupItem([html.Div([
                                        html.A(html.P(apple_news[2]['title'],style=news_style),
                                        href=(f"{apple_news[2]['link']}")),
                                        html.A(html.P(apple_news[2]['published'],style=news_style))
                                    ])
                                  ],color='gray'),
                dbc.ListGroupItem([html.Div([
                                        html.A(html.P(apple_news[3]['title'],style=news_style),
                                        href=(f"{apple_news[3]['link']}")),
                                        html.A(html.P(apple_news[3]['published'],style=news_style))
                                    ])
                                  ],color='gray'),
                dbc.ListGroupItem([html.Div([
                                        html.A(html.P(apple_news[4]['title'],style=news_style),
                                        href=(f"{apple_news[4]['link']}")),
                                        html.A(html.P(apple_news[4]['published'],style=news_style))
                                    ])
                                  ],color='gray'),
            ],flush=True)        
        ],style=news_style_b)

# if __name__ == '__main__':
#     app.run_server(debug=True,mode='inline')

In [152]:
# list_group

In [210]:
stock_df[stock_df['ticker']=='CSCO']

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,sector,ticker,SMA_5,SMA_15,SMA_ratio,15MA,SD,upperband,lowerband
0,1990-02-16,0.000000,0.059298,0.054786,0.057364,940636800.0,0.0,0.0,Technology,CSCO,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1990-02-20,0.000000,0.059297,0.055430,0.059297,151862400.0,0.0,0.0,Technology,CSCO,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1990-02-21,0.000000,0.058653,0.056075,0.058008,70531200.0,0.0,0.0,Technology,CSCO,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1990-02-22,0.000000,0.060586,0.058653,0.058653,45216000.0,0.0,0.0,Technology,CSCO,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1990-02-23,0.000000,0.059297,0.058008,0.058331,44697600.0,0.0,0.0,Technology,CSCO,0.058331,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7880,2021-05-28,53.180000,53.220001,52.840000,52.900002,14863500.0,0.0,0.0,Technology,CSCO,53.086000,52.802000,0.994650,52.802000,0.431446,53.664893,51.939108
7881,2021-06-01,52.959999,53.320000,52.430000,52.619999,15741700.0,0.0,0.0,Technology,CSCO,52.932000,52.766000,0.996864,52.766000,0.421863,53.609727,51.922273
7882,2021-06-02,52.650002,52.990002,52.400002,52.959999,14529300.0,0.0,0.0,Technology,CSCO,52.862000,52.774667,0.998348,52.774667,0.424599,53.623864,51.925469
7883,2021-06-03,52.799999,53.560001,52.619999,53.330002,17593100.0,0.0,0.0,Technology,CSCO,52.946000,52.890000,0.998942,52.890000,0.299165,53.488331,52.291669
